In [1]:
#128 bins below 0
#128 bins above 0
#1 bin below -1
#1 bin above 1
bins = [i / 128 for i in range(-128, 129)]
print(bins)

[-1.0, -0.9921875, -0.984375, -0.9765625, -0.96875, -0.9609375, -0.953125, -0.9453125, -0.9375, -0.9296875, -0.921875, -0.9140625, -0.90625, -0.8984375, -0.890625, -0.8828125, -0.875, -0.8671875, -0.859375, -0.8515625, -0.84375, -0.8359375, -0.828125, -0.8203125, -0.8125, -0.8046875, -0.796875, -0.7890625, -0.78125, -0.7734375, -0.765625, -0.7578125, -0.75, -0.7421875, -0.734375, -0.7265625, -0.71875, -0.7109375, -0.703125, -0.6953125, -0.6875, -0.6796875, -0.671875, -0.6640625, -0.65625, -0.6484375, -0.640625, -0.6328125, -0.625, -0.6171875, -0.609375, -0.6015625, -0.59375, -0.5859375, -0.578125, -0.5703125, -0.5625, -0.5546875, -0.546875, -0.5390625, -0.53125, -0.5234375, -0.515625, -0.5078125, -0.5, -0.4921875, -0.484375, -0.4765625, -0.46875, -0.4609375, -0.453125, -0.4453125, -0.4375, -0.4296875, -0.421875, -0.4140625, -0.40625, -0.3984375, -0.390625, -0.3828125, -0.375, -0.3671875, -0.359375, -0.3515625, -0.34375, -0.3359375, -0.328125, -0.3203125, -0.3125, -0.3046875, -0.296875,

In [2]:
# bins.insert(0, float(-inf))
# bins.append(float(inf))
# print(bins)

In [3]:
bin_ranges = []
for i in range(len(bins) - 1):
    bin_ranges.append((bins[i], bins[i + 1]))

print(bin_ranges)

[(-1.0, -0.9921875), (-0.9921875, -0.984375), (-0.984375, -0.9765625), (-0.9765625, -0.96875), (-0.96875, -0.9609375), (-0.9609375, -0.953125), (-0.953125, -0.9453125), (-0.9453125, -0.9375), (-0.9375, -0.9296875), (-0.9296875, -0.921875), (-0.921875, -0.9140625), (-0.9140625, -0.90625), (-0.90625, -0.8984375), (-0.8984375, -0.890625), (-0.890625, -0.8828125), (-0.8828125, -0.875), (-0.875, -0.8671875), (-0.8671875, -0.859375), (-0.859375, -0.8515625), (-0.8515625, -0.84375), (-0.84375, -0.8359375), (-0.8359375, -0.828125), (-0.828125, -0.8203125), (-0.8203125, -0.8125), (-0.8125, -0.8046875), (-0.8046875, -0.796875), (-0.796875, -0.7890625), (-0.7890625, -0.78125), (-0.78125, -0.7734375), (-0.7734375, -0.765625), (-0.765625, -0.7578125), (-0.7578125, -0.75), (-0.75, -0.7421875), (-0.7421875, -0.734375), (-0.734375, -0.7265625), (-0.7265625, -0.71875), (-0.71875, -0.7109375), (-0.7109375, -0.703125), (-0.703125, -0.6953125), (-0.6953125, -0.6875), (-0.6875, -0.6796875), (-0.6796875, -0

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/expanse/lustre/projects/csd819/a3murali/cache/'

from huggingface_hub import login
login(token = "<hf_token>")

import transformers
import torch

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

print(f"PyTorch Version: {torch.__version__}")
print(f"Transformers Version: {transformers.__version__}")

from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForCausalLM, pipeline, LlamaForCausalLM

model = "ivnle/llamatales_jr_8b-lay8-hs512-hd8-33M"
tokenizer = AutoTokenizer.from_pretrained(model)


llamatales_pipeline = pipeline("text-generation", model = model, device = "cuda")

# prompts = {
#     1: "Once upon a time there was a dragon", 
#     2: "Once upon a time there was a princess", 
#     3: "Once upon a time there were two children",
#     4: "Once upon a time there was a prince",
#     5: "Once upon a time there was a frog",
#     6: "Once upon a time there was a king",
#     7: "Once upon a time there was a queen",
#     8: "Once upon a time there was a wolf",
#     9: "Once upon a time there was a genie",
#     10: "Once upon a time there was a poor boy"
# }
prompts = {1: "Once upon a time there was a dragon"}
#generation - generate stories for each prompt
sequences = llamatales_pipeline(
    prompts[1],
    do_sample = True,
    top_k = 10,
    num_return_sequences = 1,
    max_new_tokens = 512
)
generated_story = sequences[0]['generated_text']
print(generated_story)

num_tokens_generated_story = len(tokenizer.encode(generated_story))

#test - collect hidden states

config = AutoConfig.from_pretrained("/expanse/lustre/projects/csd819/a3murali/cache/models--ivnle--llamatales_jr_8b-lay8-hs512-hd8-33M/snapshots/461f50f0024efb46b94dc68cc850d12d75ecb325", output_hidden_states = True)
model = AutoModelForCausalLM.from_config(config).to('cuda')

tokenizer = AutoTokenizer.from_pretrained("/expanse/lustre/projects/csd819/a3murali/cache/models--ivnle--llamatales_jr_8b-lay8-hs512-hd8-33M/snapshots/461f50f0024efb46b94dc68cc850d12d75ecb325/", config = config)

inputs = tokenizer(generated_story, return_tensors="pt").to("cuda")

outputs = model.generate(inputs.input_ids, attention_mask=inputs["attention_mask"], do_sample = True, top_k = 10, num_return_sequences = 1, max_new_tokens = 512, eos_token_id = tokenizer.eos_token_id, pad_token_id = tokenizer.pad_token_id, return_dict_in_generate=True)

convert_hidden_states = np.array([])

convert_hidden_states = np.append(convert_hidden_states, np.array([j.detach().cpu().numpy() for i in outputs.hidden_states for j in i]))

# print("Min Value: ", np.min(np.array(convert_hidden_states)))
# print("Max Value: ", np.max(convert_hidden_states))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/a3murali/.cache/huggingface/token
Login successful
PyTorch Version: 1.13.0a0+d321be6
Transformers Version: 4.35.0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Once upon a time there was a dragon named Sam. Sam was a big, friendly dragon. He loved to play with the children in the village. But one day, the villagers asked Sam to help them. A big flood was coming and their homes were going to get wet.

Sam said, "Don't worry, I will help." He flew high above the water and saw a long bridge. He flew over the bridge and landed on the other side. But, oh no! A mean old bear was also on the other side of the bridge.

The bear was very big and scary. Sam was worried that the bear might eat him. But, to Sam's surprise, the bear said, "Hello, little dragon. I'm not going to eat you. I need your help." The bear told Sam that he was lost too. He had flown over the river and couldn't find his way home.

Sam and the bear became friends. Sam helped the bear to find his way home. The villagers saw this and were happy. They said, "We should be nice to everyone, even if they look different." Sam and the bear said together, "We are friends now." The dragon, Sa

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/scratch/a3murali/job_36208149/ipykernel_2855319/174019619.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  convert_hidden_states = np.append(convert_hidden_states, np.array([j.detach().cpu().numpy() for i in outputs.hidden_states for j in i]))


ValueError: could not broadcast input array from shape (258,512) into shape (1,)

In [13]:
print(type(convert_hidden_states))
print(convert_hidden_states.shape)
hidden_vals = convert_hidden_states.flatten()
print(hidden_vals.shape)
bin_counts = [0 for i in range(len(bin_ranges) + 2)]
# bin_counts.append(0)
# bin_counts.append(0)
print(hidden_vals)
for i in hidden_vals:
    for j in range(len(bin_ranges)):
        if(i >= bin_ranges[j][0] and i < bin_ranges[j][1]):
            bin_counts[j + 1] += 1
    if(i < -1):
        bin_counts[0] += 1
    elif(i >= 1):
        bin_counts[len(bin_counts) - 1] += 1


# plt.hist(bin_counts, bins = 258)
# plt.show()
print(bin_counts)

<class 'numpy.ndarray'>
(3635712,)
(3635712,)
[ 0.00562103 -0.00115525  0.0048254  ...  0.9034965  -0.09795559
  0.28680092]


KeyboardInterrupt: 